# env set-up
Julia-lang 1.3.1: `PyCall` (1.91.2), `DataFrames` (0.20.0), `SymPy` (1.0.18)

In [1]:
using PyCall, DataFrames, SymPy
sympy.init_printing() ## use other py3 function not implemented in native Julia-lang, http://mth229.github.io/symbolic.html
println()

## parameters & equations

In [2]:
@vars C P B
TC,
E_PV,g_P0,r_P0,m_P0,a_P0,e_P,
E_BV,g_B0,r_B0,m_B0,e_B =
symbols(
    "TC 
E_P g_P0 r_P0 m_P0 a_P0 e_P 
E_B g_B0 r_B0 m_B0 e_B",
    positive = true)

(TC, E_P, g_P0, r_P0, m_P0, a_P0, e_P, E_B, g_B0, r_B0, m_B0, e_B)

In [3]:
R = 8.31 # ideal gas constant
T_k = TC+273.15 # unit conversion from Celsius to Kelvin
E_P = E_PV*1.6e-19 # unit conversion from eV to J
E_B = E_BV*1.6e-19

g_P = g_P0*exp(-E_P/(R*T_k))
r_P = r_P0*exp(-E_P/(R*T_k))
m_P = m_P0*exp(-E_P/(R*T_k))
a_P = a_P0*exp(-E_P/(R*T_k))

g_B = g_B0*exp(-E_B/(R*T_k))
r_B = r_B0*exp(-E_B/(R*T_k))
m_B = m_B0*exp(-E_B/(R*T_k))

C_eq = (g_P*(1 -e_P) +a_P*P +m_P)*P + (g_B*(1 -e_B)*C +m_B)*B
P_eq = (g_P*e_P -a_P*P -r_P -m_P)*P
B_eq = (g_B*e_B*C -r_B -m_B)*B

  ⎛               -1.6e-19⋅E_B                -1.6e-19⋅E_B                -1.6
  ⎜            ───────────────────         ───────────────────         ───────
  ⎜            8.31⋅TC + 2269.8765         8.31⋅TC + 2269.8765         8.31⋅TC
B⋅⎝C⋅e_B⋅g_B0⋅ℯ                    - m_B0⋅ℯ                    - r_B0⋅ℯ       

e-19⋅E_B    ⎞
────────────⎟
 + 2269.8765⎟
            ⎠

## default parameter values

In [4]:
Ceq = Eq(C_eq,0)
Peq = Eq(P_eq,0)
Beq = Eq(B_eq,0)

ss = solve([Ceq,Peq,Beq],[C,P,B])
ss

2-element Array{Tuple{Sym,Sym,Sym},1}:
 (0.0, 0.0, B)                                                                                                                                                                                                                      
 ((m_B0 + r_B0)/(e_B*g_B0), (e_P*g_P0 - m_P0 - r_P0)/a_P0, -e_B*(e_P*g_P0^2 - e_P*g_P0*r_P0 - g_P0*m_P0 - g_P0*r_P0 + m_P0*r_P0 + r_P0^2)*exp(3.85078219013237e-19*(E_B - E_P)/(20.0*TC + 5463.0))/(a_P0*(-e_B*r_B0 + m_B0 + r_B0)))

In [5]:
[ss[2][1],ss[2][2],ss[2][3]]

3-element Array{Sym,1}:
                                                                                                                                                 (m_B0 + r_B0)/(e_B*g_B0)
                                                                                                                                            (e_P*g_P0 - m_P0 - r_P0)/a_P0
 -e_B*(e_P*g_P0^2 - e_P*g_P0*r_P0 - g_P0*m_P0 - g_P0*r_P0 + m_P0*r_P0 + r_P0^2)*exp(3.85078219013237e-19*(E_B - E_P)/(20.0*TC + 5463.0))/(a_P0*(-e_B*r_B0 + m_B0 + r_B0))

In [9]:
tP = solve(Eq(ss[2][3],0), [g_P,e_P,r_P,m_P,a_P])
tP

1-element Array{NTuple{5,Sym},1}:
 (g_P0*exp(-1.6e-19*E_P/(8.31*TC + 2269.8765)), (m_P0 + r_P0)/g_P0, r_P0*exp(-1.6e-19*E_P/(8.31*TC + 2269.8765)), m_P0*exp(-1.6e-19*E_P/(8.31*TC + 2269.8765)), a_P0*exp(-1.6e-19*E_P/(8.31*TC + 2269.8765)))

In [15]:
[tP[1][1],tP[1][2],tP[1][3],tP[1][4],tP[1][5]]

5-element Array{Sym,1}:
 g_P0*exp(-1.6e-19*E_P/(8.31*TC + 2269.8765))
                           (m_P0 + r_P0)/g_P0
 r_P0*exp(-1.6e-19*E_P/(8.31*TC + 2269.8765))
 m_P0*exp(-1.6e-19*E_P/(8.31*TC + 2269.8765))
 a_P0*exp(-1.6e-19*E_P/(8.31*TC + 2269.8765))